In [ ]:
!pip install cantera

from tqdm import tqdm
import matplotlib.pyplot as plt
import cantera as ct
import pandas as pd
import numpy as np
import os

In [ ]:
fuel = {"H2": 1.0}  # X_{fuel}
phis = np.arange(0.2, 6, 0.2)
mechanism = "h2o2.yaml"
transport_models = ["multicomponent", "mixture-averaged", "unity-Lewis-number"]
oxidizer = {"O2": 1.0, "N2": 3.76}  # X_{oxidizer}
T = 300.0
p = ct.one_atm

cases = {}
for phi in phis:
    for transport_model in transport_models:
        cases[f"{phi}_{mechanism}_{transport_model}"] = {}
        cases[f"{phi}_{mechanism}_{transport_model}"]["phi"] = phi
        cases[f"{phi}_{mechanism}_{transport_model}"]["mechanism"] = mechanism
        cases[f"{phi}_{mechanism}_{transport_model}"][
            "transport_model"
        ] = transport_model

df_flame_props = pd.DataFrame(
    0.0,
    columns=["phi", "transport_model", "s_L"],
    index=[],
)

if not os.path.isdir("Data"):
    os.mkdir("Data")
if not os.path.isdir("Data/Flames_H2"):
    os.mkdir("Data/Flames_H2")

print("Solving or reading flames...")
for case in tqdm(cases):
    if os.path.isfile(f"Data/Flames_H2/{case}.csv"):
        df_flame = pd.read_csv(f"Data/Flames_H2/{case}.csv")
        cases[case]["flame"] = df_flame
        s_L = df_flame.velocity.iloc[0]
        df_flame_props.loc[len(df_flame_props)] = [
            cases[case]["phi"],
            cases[case]["transport_model"],
            s_L,
        ]

    else:
        try:
            gas = ct.Solution(cases[case]["mechanism"])
            gas.TPX = T, p, fuel
            gas.set_equivalence_ratio(cases[case]["phi"], fuel, oxidizer)
            flame = ct.FreeFlame(gas, width=0.3)
            flame.transport_model = cases[case]["transport_model"]

            flame.energy_enabled = True
            flame.set_refine_criteria(ratio=2, slope=0.05, curve=0.05, prune=0.01)

            flame.set_max_jac_age(10, 10)
            flame.set_time_step(1e-8, [10, 20, 40, 80, 100, 100, 150])
            flame.max_time_step_count = 5000    

            # Set steady-state tolerance to relative and absolute tolerances of 1e-15
            flame.flame.set_steady_tolerances(default=[1.0e-5, 1.0e-9])
            flame.flame.set_transient_tolerances(default=[1.0e-5, 1.0e-9])
            flame.solve(loglevel=0, auto=True)

            # Save flames in "Data/Flames_CH4"
            df_flame = flame.to_pandas()
            df_flame.to_csv(f"Data/Flames_H2/{case}.csv", index=False)
            cases[case]["flame"] = df_flame
            s_L = df_flame.velocity.iloc[0]
            df_flame_props.loc[len(df_flame_props)] = [
                cases[case]["phi"],
                cases[case]["transport_model"],
                s_L,
            ]

        except:
            print(f"Error for case {case}")

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(4, 4))
colors = {
    "multicomponent": "C0",
    "mixture-averaged": "C1",
    "unity-Lewis-number": "C2",
}
linestyles = {
    "multicomponent": "-",
    "mixture-averaged": "--",
    "unity-Lewis-number": "-.",
}

for transport_model in transport_models:
    df_flame_props_transport_model = df_flame_props[
        df_flame_props["transport_model"] == transport_model
    ]
    ax.plot(
        df_flame_props_transport_model["phi"],
        df_flame_props_transport_model["s_L"],
        color=colors[transport_model],
        label=transport_model,
        linestyle=linestyles[transport_model],
    )

ax.set_xlabel(r"$\phi$ $[-]$")
ax.set_ylabel(r"$s_L$ $[m/s]$")
ax.legend()
ax.grid(True)
ax.title.set_text(f"H2")
plt.show()

In [ ]:
errors = {}
df_ref = df_flame_props[df_flame_props["transport_model"] == "multicomponent"].reset_index(drop=True)
for transport_model in transport_models:
    df_flame_props_transport_model = df_flame_props[
        df_flame_props["transport_model"] == transport_model
    ].reset_index(drop=True)
    errors[transport_model] = np.abs(
        (df_flame_props_transport_model["s_L"] - df_ref["s_L"]) / df_ref["s_L"]
    )

fig, ax = plt.subplots(1, 1, figsize=(4, 4))
for i_transport, transport_model in enumerate(transport_models[1:]):
    ax.plot(
        phis,
        errors[transport_model],
        color=colors[transport_model],
        label=transport_model,
        linestyle=linestyles[transport_model],
    )


ax.set_xlabel(r"$\phi$ $[-]$")
ax.set_ylabel(r"error($s_L$) $[-]$")
ax.legend()
ax.grid()
plt.show()